In [1]:
import re
import pandas as pd
import numpy as np
import os
import sqlite3
import datetime

In [2]:
#database location
db_location = 'crawlers/usedcarsni/database.db'


In [3]:
def read_database(db_path):
    conn = sqlite3.connect(db_path)
    query = "SELECT * FROM uncleaned_used_cars"
    df = pd.read_sql_query(query, conn)
    conn.close()

    today = datetime.date.today()
    yesterday = today - datetime.timedelta(days=1)

    df['datestamp'] = pd.to_datetime(df['datestamp'])
    df_today = df[df['datestamp'].dt.date == today]
    df_yesterday = df[df['datestamp'].dt.date == yesterday]

    return df_today, df_yesterday

In [4]:
#create dataframe from database
df_today, df_yesterday = read_database(db_location)
print(df_today.info())
print(df_yesterday.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17264 entries, 51360 to 68623
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   title         17264 non-null  object        
 1   price         17081 non-null  object        
 2   year          17264 non-null  object        
 3   milage        17264 non-null  object        
 4   transmission  17264 non-null  object        
 5   fuel_type     17264 non-null  object        
 6   body_style    17264 non-null  object        
 7   engine_size   17264 non-null  object        
 8   doors         17264 non-null  object        
 9   location      17264 non-null  object        
 10  link          17264 non-null  object        
 11  datestamp     17264 non-null  datetime64[ns]
 12  timestamp2    17264 non-null  object        
dtypes: datetime64[ns](1), object(12)
memory usage: 1.1+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 17073 entries, 34287 t

In [5]:
def extract_number(link):
    pattern = r'(\d{9})'  # This pattern matches a sequence of 9 digits.
    match = re.search(pattern, link)
    if match:
        return match.group(1)
    else:
        return None

def extract_unoque_string(link):
    unique_string = link.split("/")[-1].split("?")[0]
    return unique_string

In [6]:
#apply extract_number function to the link column
df_today['ad_id'] = df_today['link'].apply(extract_number)
df_yesterday['ad_id'] = df_yesterday['link'].apply(extract_number)

#apply extract_unoque_string function to the link column
df_today['unique_string'] = df_today['link'].apply(extract_unoque_string)
df_yesterday['unique_string'] = df_yesterday['link'].apply(extract_unoque_string)

section to get only unique rows

In [7]:

def check_csv_file2(df_today, df_yesterday):
  
    list_today = df_today['ad_id'].tolist()
    list_yestrerday = df_yesterday['ad_id'].tolist()

    unique_values = set(list_today) - set(list_yestrerday)
    return unique_values


In [8]:
output = check_csv_file2(df_today, df_yesterday)

In [9]:
def save_list_to_database_with_duplicates(db_path, table_name, duplicate_table_name, data_list):
    conn = sqlite3.connect(db_path)
    c = conn.cursor()

    # create table if it doesn't already exist
    c.execute(f"CREATE TABLE IF NOT EXISTS {table_name} (id INTEGER PRIMARY KEY, datestamp DATE)")
    c.execute(f"CREATE TABLE IF NOT EXISTS {duplicate_table_name} (id INTEGER PRIMARY KEY, datestamp DATE)")
    
    # get today's date for the datestamp value
    today = datetime.date.today()

    # insert data into table with today's date as the datestamp value
    for item in data_list:
        try:
            c.execute(f"INSERT INTO {table_name} (id, datestamp) VALUES (?, ?)", (item, today))
        except sqlite3.IntegrityError:
            # if the ID already exists, save it to the duplicate table instead
            c.execute(f"INSERT INTO {duplicate_table_name} (id, datestamp) VALUES (?, ?)", (item, today))

    # commit changes and close connection
    conn.commit()
    conn.close()

In [10]:
save_list_to_database_with_duplicates(db_location, 'unique_ids', 'duplicate_ids', output)

In [11]:
print(len(output))

759
